In [28]:
import requests
import nltk
import pandas as pd
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [29]:
#
def positive_words_func():
    with open("positive-words.txt", "r") as file:
        data = file.read()  # Read entire contents as a string
        Positive_words = [word.lower() for word in data.split()]
    return Positive_words

In [30]:
#
def negative_words_func():
    with open("negative-words.txt", "r") as file:
        data = file.read()  # Read entire contents as a string
        Negative_words = [word.lower() for word in data.split()]
    return Negative_words

In [53]:
#
def currencies_stopwords_func():
    currencies_stopwords = []
    keyword = "|"
    count = 0
    temp = ""
    with open("StopWords_Currencies.txt", "r") as file:
        for line in file:
            for currencies in line:
                if currencies in "|":
                    currencies_stopwords.append(temp.strip().lower())
                    temp = ""
                    break
                else:
                    temp = temp + currencies
    return currencies_stopwords

In [51]:
#
def auditor_stopwords_func():
    with open("StopWords_Auditor.txt", "r") as file:
        data = file.read()  # Read entire contents as a string
        Auditors_stopwords = [word.lower() for word in data.split()]
    return Auditors_stopwords

In [33]:
#
def stopwords_function(text, currencies_stopwords, Auditors_stopwords):
    stop_words = set(stopwords.words('english'))
    
    # Tokenize the text (split into words)
    words = text.split()

    # Remove stop words from the list of words
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [word for word in filtered_words if word.lower() not in currencies_stopwords]
    filtered_words = [word for word in filtered_words if word.lower() not in Auditors_stopwords]

    return filtered_words

In [34]:
#
def Subjectivity_score(positive_score, negative_score, word_len):
    # Subjectivity Score
    subjectivity_score = (positive_score + negative_score)/ ((word_len) + 0.000001)
    return subjectivity_score

In [35]:
def Polarity_score(positive_score, negative_score):
    # Polarity Score
    polarity_score = (positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)
    return polarity_score

In [36]:
#
def score_count(filtered_words, words_list):
    score = 0
    for i in filtered_words:
        if i in words_list:
            score += 1
    return score

In [37]:
#
def fog_index_func(avg_sentence_len, perc_complex_words):
    # fog index
    fog_index = 0.4 * (avg_sentence_len + perc_complex_words)
    return fog_index

In [38]:
#
def perc_complex_word_func(total_complex_words, total_words):
    # percentage of complex words
    perc_complex_words = total_complex_words / total_words
    return perc_complex_words

In [39]:
#
def avg_sentence_length(total_words, num_sentences):
    # average number of words per sentence
    # average sentence length
    avg_sentence_len = total_words / num_sentences
    return avg_sentence_len

In [40]:
#
def count_sentences_nltk(text):
    # Check NLTK version (optional)
    if nltk.__version__ >= '3.6':
        print("Using nltk.sent_tokenize.PunktSentenceTokenizer (recommended)")
        sentences = nltk.sent_tokenize(text)  # Recommended method for NLTK >= 3.6
        return len(sentences)

In [41]:
#
def complex_word_count_func(words1):
    # complex word count
    total_complex_words = 0
    for i in words1:
        if words1[i][2] / words1[i][0] > 2:
            total_complex_words += words1[i][0]
    return total_complex_words

In [42]:
#
def count_personal_pronoun(words1):
    # count personal pronouns

    # Define personal pronouns (case-insensitive)
    personal_pronouns = [
          "I", "me", "my", "mine", "myself",
          "you", "your", "yours", "yourself",
          "he", "him", "his", "himself",
          "she", "her", "hers", "herself",
          "it", "its", "itself",
          "we", "us", "our", "ours", "ourselves",
          "they", "them", "their", "theirs", "themselves"
      ]

    total_personal_nouns = 0
    for i in words1:
        if i in personal_pronouns:
            total_personal_nouns += words1[i][0]

    return total_personal_nouns

In [43]:
#
def avg_word_len_func(total_char, total_words):
    # average word length
    avg_word_len = total_char / total_words
    return avg_word_len

In [44]:
#
def total_count(words1):
    # total characters
    total_char = 0
    total_words = 0
    for i in words1: 
        total_words = total_words + words1[i][0]
        total_char = total_char + (words1[i][0]*words1[i][1])
    return [total_char, total_words]

In [45]:
#
def count_total_words(word_tokens):
    # Count total number of characters in each word
    words1 = {}
    punctuation1 = {}
    vowels = ["a", "e", "i", "o", "u"]
    for i in word_tokens:
        if len(i) == 0:
            continue
        elif 'a' <= i[0] <= 'z' or 'A' <= i[0] <= 'Z':
            if len(i) > 1 and i[0] == "U" and i[1] == "S":
                pass
            else:
                i = i.lower()
            if i not in words1:
                words1[i] = [1, len(i), 0]
            else:
                words1[i][0] = words1[i][0] + 1
            words1[i][2] += count_syllables(i)
        else:
            if i not in punctuation1:
                punctuation1[i] = [1, len(i)]
            else:
                punctuation1[i][0] += 1
    return words1

In [46]:
def final_syllable_count(words1, total_words):
    syllable_count = 0
    for i in words1:
        syllable_count  = syllable_count + words1[i][2]
    return syllable_count

In [47]:
#
def count_syllables(text):
  vowels = "aeiou"
  exceptions = {"es": True, "ed": True}  # Words ending with these are not syllables

  syllable_count = 0
  last_char_vowel = False

  for char in text:
    if char in vowels:
      # If the current character is a vowel and the last character wasn't,
      # it's likely a new syllable (except for exceptions)
      if not last_char_vowel:
        syllable_count += 1
        if char in exceptions and len(text) > 2 and text[-3] not in vowels:
          # Handle exceptions like "love" and "loved" (but not "lovedd")
          syllable_count -= 1
      last_char_vowel = True
    else:
      last_char_vowel = False

  return syllable_count

In [48]:
#
def tokenize_the_text(text):
    # Tokenize the text into words
    word_tokens = []
    temp = ""
    for i in text:
        if 'a' <= i <= 'z' or 'A' <= i <= 'Z':
            temp = temp + i
        elif '0' <= i <= '9':
            temp = temp + i
        elif i == " ":
            word_tokens.append(temp)
            temp = ""
        else:
            word_tokens.append(temp)
            word_tokens.append(i)
            temp = ""
    return word_tokens

In [49]:
#
def web_crawler_func(URL):

    # Send a GET request to the specified URL and store the response in 'resp'
    resp = requests.get(URL)

    # Parse the HTML content of the response using BeautifulSoup
    soup = BeautifulSoup(resp.text, 'html.parser')

    #print(soup)

    entry_title = soup.find('h1', class_='entry-title')

    header_title = entry_title.get_text()

    specific_div = soup.find('div', class_='td-post-content tagdiv-type')

    # Get the content of the specific <div>
    specific_div_content = specific_div.get_text()


    def remove_text_after_word(text, word):
        index = text.find(word)
        if index != -1:
            return text[:index]
        return text


    specific_div_content = remove_text_after_word(specific_div_content, "Blackcoffer Insights")

    text = header_title + specific_div_content
    return text

In [ ]:
# Specify the path to your Excel file
file_path = "blackcoffer_dataset2.xlsx"

# Read the data from the first sheet (index 0) using pandas.read_excel()
data = pd.read_excel(file_path)

# Create a output DataFrame
output_data = {'URL id': [], 'POSITIVE SCORE': [], 'NEGATIVE SCORE': [], 'POLARITY SCORE': [], 'SUBJECTIVITY SCORE': [], 'AVG SENTENCE LENGTH': [], 'PERCENTAGE OF COMPLEX WORDS': [], 'FOG INDEX': [], 'AVG NUMBER OF WORDS PER SENTENCE': [], 'COMPLEX WORD COUNT': [], 'WORD COUNT': [], 'SYLLABLE PER WORD': [], 'PERSONAL PRONOUNS': [], 'AVG WORD LENGTH': []}
df = pd.DataFrame(output_data)

for i in range(len(data['URL'])):
    text = web_crawler_func(data['URL'][i])
    word_tokens = tokenize_the_text(text)
    words1 = count_total_words(word_tokens)
    total_char, total_words = total_count(words1)
    avg_word_len = avg_word_len_func(total_char, total_words)
    total_personal_nouns = count_personal_pronoun(words1)
    total_complex_words = complex_word_count_func(words1)
    num_sentences = count_sentences_nltk(text)
    avg_sentence_len = avg_sentence_length(total_words, num_sentences)
    perc_complex_words = perc_complex_word_func(total_complex_words, total_words)
    fog_index = fog_index_func(avg_sentence_len, perc_complex_words)
    syllable_count = final_syllable_count(words1, total_words)
    Auditor_stopwords = auditor_stopwords_func()
    currencies_stopwords = currencies_stopwords_func()
    filtered_words = stopwords_function(text, currencies_stopwords, Auditor_stopwords)
    Positive_words = positive_words_func()
    Negative_words = negative_words_func()
    positive_score = score_count(filtered_words, Positive_words)
    negative_score = score_count(filtered_words, Negative_words)
    subjectivity_score = Subjectivity_score(positive_score, negative_score, len(filtered_words))
    polarity_score = Polarity_score(positive_score, negative_score)

    new_row = {'URL id': data['URL_ID'],
               'POSITIVE SCORE': positive_score,
               'NEGATIVE SCORE': negative_score,
               'POLARITY SCORE': polarity_score,
               'SUBJECTIVITY SCORE': subjectivity_score,
               'AVG SENTENCE LENGTH': avg_sentence_len,
               'PERCENTAGE OF COMPLEX WORDS': perc_complex_words,
               'FOG INDEX': fog_index,
               'AVG NUMBER OF WORDS PER SENTENCE': avg_sentence_len,
               'COMPLEX WORD COUNT': total_complex_words, 
               'WORD COUNT': len(filtered_words),
               'SYLLABLE PER WORD': syllable_count,
               'PERSONAL PRONOUNS': total_personal_nouns,
               'AVG WORD LENGTH': avg_word_len
              }

    df = df.append(new_row, ignore_index=True)

print(df)

# Save the DataFrame to an Excel file
df.to_excel("output_data_harshal_jain.xlsx")

Using nltk.sent_tokenize.PunktSentenceTokenizer (recommended)


C:\Users\DELL\AppData\Local\Temp\ipykernel_11140\1421133981.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


Using nltk.sent_tokenize.PunktSentenceTokenizer (recommended)


C:\Users\DELL\AppData\Local\Temp\ipykernel_11140\1421133981.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


Using nltk.sent_tokenize.PunktSentenceTokenizer (recommended)


...

C:\Users\DELL\AppData\Local\Temp\ipykernel_11140\1421133981.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


Using nltk.sent_tokenize.PunktSentenceTokenizer (recommended)


C:\Users\DELL\AppData\Local\Temp\ipykernel_11140\1421133981.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


Using nltk.sent_tokenize.PunktSentenceTokenizer (recommended)


C:\Users\DELL\AppData\Local\Temp\ipykernel_11140\1421133981.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


In [55]:
new_row

{'URL id': 'hello',
 'POSITIVE SCORE': 41,
 'NEGATIVE SCORE': 5,
 'POLARITY SCORE': 0.7826086786389418,
 'SUBJECTIVITY SCORE': 0.07044410402688499,
 'AVG SENTENCE LENGTH': 15.512820512820513,
 'PERCENTAGE OF COMPLEX WORDS': 0.1396694214876033,
 'FOG INDEX': 6.260995973723247,
 'AVG NUMBER OF WORDS PER SENTENCE': 15.512820512820513,
 'COMPLEX WORD COUNT': 169,
 'WORD COUNT': 653,
 'SYLLABLE PER WORD': 1942,
 'PERSONAL PRONOUNS': 69,
 'AVG WORD LENGTH': 4.571074380165289}